## all-MiniLM-L6-v1 Hugging Face Model to ONNX


In [4]:
%pip install -q transformers transformers[onnx] transformers[sentencepiece] einops
%pip install -q torch torchaudio 
%pip install -U -q sentence-transformers


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v1  models/all-MiniLM-L6-v1

## Converter Help  

```
usage: Hugging Face Transformers ONNX exporter [-h] -m MODEL
                                               [--feature FEATURE]
                                               [--opset OPSET] [--atol ATOL]
                                               [--framework {pt,tf}]
                                               [--cache_dir CACHE_DIR]
                                               [--preprocessor {auto,tokenizer,feature_extractor,processor}]
                                               [--export_with_transformers]
                                               output

positional arguments:
  output                Path indicating where to store generated ONNX model.

options:
  -h, --help            show this help message and exit
  -m MODEL, --model MODEL
                        Model ID on huggingface.co or path on disk to load
                        model from.
  --feature FEATURE     The type of features to export the model with.
  --opset OPSET         ONNX opset version to export the model with.
  --atol ATOL           Absolute difference tolerance when validating the
                        model.
  --framework {pt,tf}   The framework to use for the ONNX export. If not
                        provided, will attempt to use the local checkpoint's
                        original framework or what is available in the
                        environment.
  --cache_dir CACHE_DIR
                        Path indicating where to store cache.
  --preprocessor {auto,tokenizer,feature_extractor,processor}
                        Which type of preprocessor to use. 'auto' tries to
                        automatically detect it.
  --export_with_transformers
                        Whether to use transformers.onnx instead of
                        optimum.exporters.onnx to perform the ONNX export. It
                        can be useful when exporting a model supported in
                        transformers but not in optimum, otherwise it is not
                        recommended.

```

In [54]:
%pip install onnxruntime-gpu numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 MB 3.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np

In [10]:
on_model = "mpt_onnx"

In [13]:
import onnxruntime as nxrun
print(nxrun.get_available_providers())
sess = nxrun.InferenceSession(f'raw-files/{on_model}/model.onnx', providers=["CUDAExecutionProvider"])

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


2023-05-23 16:00:40.380510020 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.31/attn/Trilu'
2023-05-23 16:00:40.382602265 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.30/attn/Trilu'
2023-05-23 16:00:40.384902956 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.29/attn/Trilu'
2023-05-23 16:00:40.387151006 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.28/attn/Trilu'
2023-05-23 16:00:40.389247712 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.27/attn/Trilu'
2023-05-23 16:00:40.391449754 [W:onnxruntime:, constant_folding.c

RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Exception during initialization: /onnxruntime_src/onnxruntime/core/framework/bfc_arena.cc:368 void* onnxruntime::BFCArena::AllocateRawInternal(size_t, bool, onnxruntime::Stream*, bool, onnxruntime::WaitNotificationFn) Failed to allocate memory for requested buffer of size 201326592


CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.9/attn/Trilu'
2023-05-23 16:00:40.549910479 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.8/attn/Trilu'
2023-05-23 16:00:40.552056950 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.7/attn/Trilu'
2023-05-23 16:00:40.554259745 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.6/attn/Trilu'
2023-05-23 16:00:40.556422240 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/blocks.5/attn/Trilu'
2023-05-23 16:00:40.558444636 [W:onnxruntime:, constant_folding.cc:179 ApplyImpl] Could not find a CPU kernel and hence can't constant fold Trilu node '/transformer/bloc

In [1]:
input_text = 'def fibonacci(n):'

In [3]:
input_text = "write a python funciton which takes a number, add and multiply with 100 and returns<|endoftext|>"
input_text = "python function that reads a file, and shows its content"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('models/mpt-7b-chat', trust_remote_code=True)
input_ids = tokenizer(input_text)["input_ids"]
attention_mask = tokenizer(input_text)["attention_mask"]
mask = len(attention_mask)
print(len(input_ids))
input_ids.extend([0] * (2048 - len(input_ids)))
attention_mask.extend([0] * (2048 - len(attention_mask)))

input_ids = np.asarray(input_ids, np.int64)
attention_mask = np.asarray(attention_mask, np.bool_)
# input_ids.shape, attention_mask
attention_mask

/media/ahwar/WD_Swap/mpt-7b-chat/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


7


NameError: name 'np' is not defined

In [7]:
input_name = sess.get_inputs()[0].name
input_name2 = sess.get_inputs()[1].name
input_shape = sess.get_inputs()[0].shape
input_shape2 = sess.get_inputs()[1].shape
output_name = sess.get_outputs()[0].name
output_shape = sess.get_outputs()[0].shape
# run onnx model with onnx runtime python
# result = sess.run(None, {input_name: np.ones((1, 2048), dtype=np.int64), input_name2: np.ones((1, 2048), dtype=np.bool_)})
result = sess.run(None, {input_name: np.expand_dims(input_ids, 0), input_name2: np.expand_dims(attention_mask, 0)})

# print("Output", result)

In [15]:
t = list()
for idx, i in enumerate(result[0][0]):
    top = int(np.argmax(i))
    t.append(top)
    # if idx == (mask - 1):
    #     break
        

In [16]:
generated_code = tokenizer.decode(t, skip_special_tokens=False, clean_up_tokenization_spaces=False)
generated_code

'- to takes a text and converts returns the contents. on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on on o

In [20]:
np.asarray(result).shape

(1, 1, 2048, 50432)

In [6]:
!python -m transformers.onnx --opset 16 --atol 0.005 \
    --feature=causal-lm --export_with_transformers \
    --model=models/all-MiniLM-L6-v1 raw-files/minilm_onnx/

Local PyTorch model found.
Framework not requested. Using torch to export to ONNX.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at models/all-MiniLM-L6-v1 and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using framework PyTorch: 2.0.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Validating ONNX model...
	-[✓] ONNX model output names match reference model ({'logits'})
	- Validating O

In [4]:
model.eval()

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): Embedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# load model
model = AutoModelForCausalLM.from_pretrained('models/mpt-7b-chat', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('models/mpt-7b-chat', trust_remote_code=True)
x = tokenizer.encode(input_text, return_tensors='pt', max_length=320)
# x = tokenizer(input_text, return_tensors="pt")
# x["attention_mask"] = x["attention_mask"].type(torch.bool)
# x
# tokenizer
# tokenizer.encode
model.to(device='cpu:0')
model.type(torch.bfloat16)


# # single input encoding + generation
x = tokenizer.encode(input_text, return_tensors='pt', max_length=320)
y = model.generate(x)

# decoding, clean_up_tokenization_spaces=False to ensure syntactical correctness
generated_code = tokenizer.decode(y[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(generated_code)

/home/ahwar/.cache/huggingface/modules/transformers_modules/mpt-7b-chat/attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# load model
model = AutoModelForCausalLM.from_pretrained('models/mpt-7b-chat', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('models/mpt-7b-chat', trust_remote_code=True)
# tokenizer
# tokenizer.encode
model.to(device='cpu:0')
model.type(torch.bfloat16)


# # single input encoding + generation
x = tokenizer.encode(input_text, return_tensors='pt', max_length=320)
y = model.generate(x)

# decoding, clean_up_tokenization_spaces=False to ensure syntactical correctness
generated_code = tokenizer.decode(y[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(generated_code)

In [4]:
y[0]

tensor([16659,  1159,   326,  9563,   247,  1873,    13,   285,  2722,   697,
         2600,    15,   187,   187, 11202, 16659,   187,  1545,  1239,    64])

In [36]:
print(input_ids)
x

[ 1545  5713   251 42401     9    79  2262     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

tensor([[ 1545,  5713,   251, 42401,     9,    79,  2262,     0]])

In [17]:
from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

sequence_a = "This is a short sequence."
sequence_b = "This is a rather long sequence. It is at least longer than the sequence A."

encoded_sequence_a = tokenizer(sequence_a)["input_ids"]
encoded_sequence_b = tokenizer(sequence_b)["input_ids"]
encoded_sequence_a

[1552, 310, 247, 2159, 3425, 15]

In [ ]:
print(x)
y

In [ ]:
print(x.shape)
y.shape


torch.Size([1, 8])
